# Document Extraction with Azure AI Document Intelligence and Microsoft Phi-4 (Text Only)

This sample demonstrates how to extract structured data from any document using Azure AI Document Intelligence and Microsoft's Phi-4 model.

![Data Extraction](../../../../images/extraction-document-intelligence-phi.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes specific extraction instruction for the type of document, the expected JSON schema, and the Markdown content of the document
- Use the chat completions API with the Phi-4 model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Use prompt engineering techniques to instruct Phi-4 to extract structured data from a type of document using example JSON structures.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.

## Useful Tips

- Combine this technique with a [page classification](../../classification/README.md) approach to reduce the number of pages to extract from to only those that match your criteria for extraction.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **azure-ai-documentintelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **azure-ai-inference** to interface with a deployed Phi-4 serverless endpoint in Azure AI Foundry.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local components are also used:

- [**invoice**](../../modules/samples/models/invoice.py) to provide the expected structured output JSON schema for invoice documents.
- [**accuracy_evaluator**](../../modules/samples/evaluation/accuracy_evaluator.py) to evaluate the output of the extraction process with expected results.
- [**document_intelligence_confidence**](../../modules/samples/confidence/document_intelligence_confidence.py) to calculate the confidence of the extraction process based on the analysis result from the Azure AI Document Intelligence API.
- [**document_processing_result**](../../modules/samples/models/document_processing_result.py) to store the results of the extraction process as a file.
- [**stopwatch**](../../modules/samples/utils/stopwatch.py) to measure the end-to-end execution time for the extraction process.
- [**app_settings**](../../modules/samples/app_settings.py) to access environment variables from the `.env` file.

In [1]:
import sys
sys.path.append('../../modules/') # Import local modules

from IPython.display import display, Markdown
import os
import pandas as pd
from dotenv import dotenv_values
from dotenv import load_dotenv
import json
from azure.ai.inference import ChatCompletionsClient
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, DocumentContentFormat
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from concurrent.futures import ThreadPoolExecutor

from samples.app_settings import AppSettings
from samples.utils.stopwatch import Stopwatch
from samples.utils.storage_utils import create_json_file
from samples.models.document_processing_result import DataExtractionResult

from samples.models.invoice import Invoice
from samples.confidence.confidence_utils import merge_confidence_values
from samples.confidence.document_intelligence_confidence import evaluate_confidence as evaluate_di_confidence
from samples.evaluation.accuracy_evaluator import AccuracyEvaluator
from samples.evaluation.comparison import get_extraction_comparison

### Configure the Azure services

To use Azure AI Document Intelligence and a serverless model endpoint deployment, their SDKs are used to create client instances using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with Azure AI Document Intelligence. The Azure AI Inference client for the Phi model uses an API key to authenticate with the serverless endpoint.

In [2]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))
sample_path = f"{working_dir}/samples/python/extraction/text"
sample_name = "document-extraction-phi"

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

# Uses the Azure AI Inference SDK to interact with the Phi model for chat completions.
inference_client = ChatCompletionsClient(
    endpoint=settings.phi_endpoint,
    credential=AzureKeyCredential(settings.phi_primary_key),
    model=settings.gpt4o_model_deployment_name,
)

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=settings.ai_services_endpoint,
    credential=credential
)

python-dotenv could not parse statement starting at line 13


### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on the details of an [Invoice](../../../assets/invoices/invoice_1.pdf).

> **Note**: More invoice examples can be found in the [assets folder](../../../assets/invoices). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

The expected output has been defined by a human evaluating the document.

In [3]:
path = f"{working_dir}/samples/assets/invoices/"
metadata_fname = "invoice_1.json" # Change this to the file you want to evaluate
metadata_fpath = f"{path}{metadata_fname}"

with open(metadata_fpath, "r") as f:
    data = json.load(f)
    
expected = Invoice(**data['0_expected'])
pdf_fname = data['fname']
pdf_fpath = f"{path}{pdf_fname}"

invoice_evaluator = AccuracyEvaluator(match_keys=['product_code', 'description']) # Product Code and Descriptions are used to match the extracted product items data

## Extract data from the document

The following code block executes the data extraction process using Azure AI Document Intelligence and Phi-4.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Using a Phi-4 Serverless Endpoint model deployment in Azure AI Foundry and prompt engineering techniques, extract a structured data transfer object (DTO) from the content of the Markdown.

In [4]:
with Stopwatch() as di_stopwatch:
    with open(pdf_fpath, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            model_id="prebuilt-layout",
            body=f,
            output_content_format=DocumentContentFormat.MARKDOWN,
            content_type="application/pdf"
        )
        
    result: AnalyzeResult = poller.result()

markdown = result.content

In [5]:
system_prompt = f""""You are an AI assistant that extracts data from documents and returns them as structured JSON objects."""

In [6]:
# Prepare the user content for the Phi inference API including any specific details for processing this type of document, and the document text.
user_content = []

In [7]:
user_text_prompt = f"""Extract the data from this invoice. 
- If a value is not present, provide null.
- Dates should be in the format YYYY-MM-DD.
- Strictly use the following JSON schema: {json.dumps(Invoice.model_json_schema())}
- ONLY return the JSON object. DO NOT return as a JSON markdown code block. DO NOT include any other detail in your response."""

user_content.append({
    "type": "text",
    "text": user_text_prompt
})

user_content.append({
    "type": "text",
    "text": markdown
})

In [8]:
with Stopwatch() as phi_stopwatch:
    completion = inference_client.complete(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        temperature=0.1,
        top_p=0.1
    )

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The Markdown representation of the document structure as determined by Azure AI Document Intelligence.
- The accuracy of the structured data extraction comparing the expected output with the output generated by Phi-4.
- The confidence score of the structured data extraction by comparing against the Azure AI Document Intelligence analysis.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the Phi-4 model.
- The side-by-side comparison of the expected output and the output generated by Phi-4.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [9]:
# Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(Markdown(markdown))

NEXGEN

Innovation drives progress


# INVOICE


<table>
<tr>
<th>Customer:</th>
<th colspan="2">Sharp Consulting</th>
<th colspan="2">Address:</th>
<th colspan="3">73 Regal Way, Leeds, LS1 5AB, UK</th>
</tr>
<tr>
<td>Delv. Date:</td>
<td>5/16/2024</td>
<td colspan="2">Invoice Number:</td>
<td colspan="2">3847193</td>
<td>Purchase Order:</td>
<td>15931</td>
</tr>
</table>


<table>
<tr>
<th>Item Code</th>
<th>Item Desc.</th>
<th>Unit Price</th>
<th>Quantity</th>
<th>Total</th>
</tr>
<tr>
<td>MA197</td>
<td>STRETCHWRAP ROLL</td>
<td>16.62</td>
<td>5</td>
<td>83.10</td>
</tr>
<tr>
<td>ST4086</td>
<td>BALLPOINT PEN MED.</td>
<td>2.49</td>
<td>10</td>
<td>24.90</td>
</tr>
<tr>
<td>JF9912413BF</td>
<td>BUBBLE FILM ROLL CL.</td>
<td>15.46</td>
<td>12</td>
<td>185.52</td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>


NOTES


<table>
<tr>
<th>Total Pcs .:</th>
<th>27</th>
<th>Total Price:</th>
<th>293.52</th>
</tr>
<tr>
<td colspan="4">Payable on or before 5/24/2024</td>
</tr>
<tr>
<td>Cust. Sig.</td>
<td>Sfr</td>
<td>Drivr. Sig.</td>
<td></td>
</tr>
<tr>
<td>Cust. Name:</td>
<td>Sarah H</td>
<td>Drivr. Name:</td>
<td>James T</td>
</tr>
</table>


<!-- PageBreak -->


<table>
<tr>
<td colspan="6">NEXGEN Innovation drives progress</td>
</tr>
<tr>
<td></td>
<td colspan="5">RETURNS</td>
</tr>
<tr>
<td>Customer:</td>
<td>Sharp Consulting</td>
<td colspan="2">Address:</td>
<td colspan="2">73 Regal Way, Leeds, LS1 5AB, UK</td>
</tr>
<tr>
<td>Coll. Date:</td>
<td>5/16/2024</td>
<td></td>
<td></td>
<td colspan="2"></td>
</tr>
<tr>
<td>Item Code</td>
<td>Item Desc.</td>
<td>Pcs.</td>
<td colspan="3">Reason</td>
</tr>
<tr>
<td>MA145</td>
<td>POSTAL TUBE BROWN</td>
<td>1</td>
<td colspan="3">This item was provided in previous order</td>
</tr>
<tr>
<td>JF7902</td>
<td>MAILBOX 25PK</td>
<td>1 7</td>
<td colspan="3">as replacement</td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td colspan="3">Not required</td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td colspan="3"></td>
</tr>
<tr>
<td colspan="6"></td>
</tr>
<tr>
<td>NOTES</td>
<td></td>
<td colspan="4"></td>
</tr>
<tr>
<td></td>
<td></td>
<td colspan="4"></td>
</tr>
<tr>
<td></td>
<td colspan="5"></td>
</tr>
<tr>
<td></td>
<td colspan="5"></td>
</tr>
<tr>
<td colspan="2"></td>
<td></td>
<td colspan="3"></td>
</tr>
<tr>
<td></td>
<td></td>
<td colspan="4"></td>
</tr>
<tr>
<td>Total Pcs .:</td>
<td>2</td>
<td></td>
<td colspan="3"></td>
</tr>
<tr>
<td colspan="6"></td>
</tr>
<tr>
<td>Cust. Sig.</td>
<td>8</td>
<td></td>
<td colspan="2">Drivr. Sig.</td>
<td>&amp;</td>
</tr>
<tr>
<td>Cust. Name:</td>
<td>Sarah H</td>
<td></td>
<td colspan="2">Drivr. Name:</td>
<td>James T</td>
</tr>
</table>


In [13]:
# Gets the JSON response from the completion and converts it to an Invoice object.
response_json = completion.choices[0].message.content

# Remove any JSON markdown code block formatting from the response, if present.
response_json = response_json.replace("```json", "").replace("```", "").strip()

print(response_json)

invoice = Invoice.model_validate(json.loads(response_json))

expected_dict = expected.model_dump()
invoice_dict = invoice.model_dump()

{
  "customer_name": "Sharp Consulting",
  "customer_tax_id": null,
  "customer_address": "73 Regal Way, Leeds, LS1 5AB, UK",
  "shipping_address": "73 Regal Way, Leeds, LS1 5AB, UK",
  "purchase_order": "15931",
  "invoice_id": "3847193",
  "invoice_date": "2024-05-16",
  "due_date": "2024-05-24",
  "vendor_name": "NEXGEN",
  "vendor_address": "NEXGEN Innovation drives progress",
  "vendor_tax_id": null,
  "remittance_address": null,
  "subtotal": 293.52,
  "total_discount": null,
  "total_tax": null,
  "invoice_total": 293.52,
  "payment_term": null,
  "items": [
    {
      "product_code": "MA197",
      "description": "STRETCHWRAP ROLL",
      "quantity": 5,
      "tax": null,
      "unit_price": 16.62,
      "total": 83.10
    },
    {
      "product_code": "ST4086",
      "description": "BALLPOINT PEN MED.",
      "quantity": 10,
      "tax": null,
      "unit_price": 2.49,
      "total": 24.90
    },
    {
      "product_code": "JF9912413BF",
      "description": "BUBBLE FILM RO

ValidationError: 11 validation errors for Invoice
customer_address
  Input should be a valid dictionary or instance of InvoiceAddress [type=model_type, input_value='73 Regal Way, Leeds, LS1 5AB, UK', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
shipping_address
  Input should be a valid dictionary or instance of InvoiceAddress [type=model_type, input_value='73 Regal Way, Leeds, LS1 5AB, UK', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
vendor_address
  Input should be a valid dictionary or instance of InvoiceAddress [type=model_type, input_value='NEXGEN Innovation drives progress', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
subtotal
  Input should be a valid dictionary or instance of InvoiceCurrency [type=model_type, input_value=293.52, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type


In [11]:
# Determines the accuracy of the extracted data against the expected values.
accuracy = invoice_evaluator.evaluate(expected=expected_dict, actual=invoice_dict)

NameError: name 'expected_dict' is not defined

In [ ]:
# Determines the confidence of the extracted data against the expected values using the result of the Azure AI Document Intelligence pre-built layout analysis.
confidence = evaluate_di_confidence(invoice_dict, result)

In [ ]:
# Gets the total execution time of the data extraction process.
total_elapsed = di_stopwatch.elapsed + phi_stopwatch.elapsed

# Gets the prompt tokens and completion tokens from the completion response.
prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

In [ ]:
# Save the output of the data extraction result.
extraction_result = DataExtractionResult(invoice_dict, confidence, accuracy, prompt_tokens, completion_tokens, total_elapsed)

create_json_file(f"{sample_path}/{sample_name}.{pdf_fname}.json", extraction_result)

In [12]:
# Display the outputs of the data extraction process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Confidence": f"{confidence['_overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Document Intelligence Execution Time": f"{di_stopwatch.elapsed:.2f} seconds",
        "Phi Execution Time": f"{phi_stopwatch.elapsed:.2f} seconds",
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens
    }
])

display(df)
display(get_extraction_comparison(expected_dict, invoice_dict, confidence, accuracy['accuracy']))

NameError: name 'accuracy' is not defined